In [3]:
# Implementation by williamchangTW
# Data mining Project 1 
# Algo : Apriori algorithm
# Data generator : IBM Quest Synthetic Data Generator
# - optiones : -trans 0.1 -litems 0.01

import pandas as pd

class Apriori():
    def __init__(self):
        pass

    '''
    关联分析的目标包括两项：发现频繁项集和发现关联规则
    '''

    '''
    频繁项集：{}
        对于包含N种物品的数据集共有2^N-1种项集组合。
        支持度(support)
            一个项集的支持度被定义为数据集中包含该项集的记录所占的比例。
        Apriori算法：如果某个项集是频繁的，那么它的所有子集也是频繁的。
        如果一个项集是非频繁集，那么它的所有超集也是非频繁集。
    '''

    def _createC1(self, dataSet):
        C1 = []
        for transaction in dataSet:
            for item in transaction:
                if [item] not in C1:
                    C1.append([item])
        C1.sort()
        return map(frozenset, C1) # use frozen set so we can use it as a key in a dict

    def _scanD(self, D, Ck, minSupport=0.5):
        ssCnt = {}
        for tid in D:
            for can in Ck:
                if can.issubset(tid):
                    if can in ssCnt:
                        ssCnt[can] += 1
                    else:
                        ssCnt[can] = 1
                    # if can not in ssCnt:
                    #     ssCnt[can] = 0
                    # ssCnt[can] += 1
        # print ssCnt
        numItems = len(D)
        retList = []
        supportK = {}
        for key in ssCnt:
            support = ssCnt[key]/float(numItems) # 计算支持度
            if support >= minSupport:
                retList.append(key)
            supportK[key] = support
        return retList, supportK

    def aprioriGen(self, Lk, k): # k>=2
        retList = []
        lenLk = len(Lk)
        for i in range(lenLk):
            for j in range(i+1, lenLk):
                L1 = list(Lk[i])[:k-2]
                L2 = list(Lk[j])[:k-2]
                L1.sort()
                L2.sort()
                if L1 == L2: # if first k-2 elements are equal. when k is 3, {0,1},{0,2},{1,2}→{0,1}U{0,2}→{0,1,2}
                    retList.append(Lk[i] | Lk[j])
        return retList

    def apriori(self, dataSet, minSupport=0.5): # minSupport 最小支持度
        D = map(set, dataSet) # 转换为集合set
        C1 = self._createC1(dataSet) # 创建C1，转换为集合frozenset
        L1, supp1 = self._scanD(D, C1, minSupport) # 基于C1和minSupport创建L1
        L = []
        supportData = {}
        L.append(L1)
        supportData.update(supp1)
        k = 2
        while len(L[k-2]) > 1:
            Ck = self.aprioriGen(L[k-2], k) # 创建Ck
            Lk, suppK = self._scanD(D, Ck, minSupport) # 基于Ck和minSupport创建Lk
            L.append(Lk)
            supportData.update(suppK)
            k += 1
        return L, supportData

    '''
    关联规则：→
        可信度(confidence)：也称置信度
            可信度(尿布→葡萄酒) = 支持度({尿布,葡萄酒})/支持度({尿布})
        如果某条规则并不满足最小可信度要求，那么该规则的所有子集也不会满足最小可信度要求。
    '''

    def _calcConf(self, freqSet, H, supportData, brl, minConf=0.7): # H为出现在右部的规则列表，如{0},{1}
        prunedH = []
        for conseq in H:
            conf = supportData[freqSet]/supportData[freqSet-conseq] # 计算可信度
            if conf >= minConf:
                print freqSet-conseq, '-->', conseq, 'conf:', conf
                brl.append((freqSet-conseq, conseq, conf))
                prunedH.append(conseq)
        return prunedH

    def _rulesFromConseq(self, freqSet, H, supportData, brl, minConf=0.7): # H为出现在右部的规则列表，如{0},{1}
        m = len(H[0])
        if len(freqSet) > (m+1):
            Hmp1 = self.aprioriGen(H, m+1) # 合并规则
            Hmp = self._calcConf(freqSet, Hmp1, supportData, brl, minConf) # Hmp为出现在右部的合并规则列表，如{0,1}
            if len(Hmp) > 1: # 如果规则列表长度大于1，则进一步合并
                self._rulesFromConseq(freqSet, Hmp, supportData, brl, minConf)

    def generateRules(self, L, supportData, minConf=0.7): # minConf 最小可信度
        bigRuleList = []
        for i in range(1, len(L)): # 从包含两个或者更多元素的项集开始规则构建过程
            for freqSet in L[i]:
                H1 = [frozenset([item]) for item in freqSet] # 构建只包含单个元素的列表，即出现在规则右部的规则列表，如{0},{1}
                if i > 1:
                    self._rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf) # 生成候选规则
                else:
                    self._calcConf(freqSet, H1, supportData, bigRuleList, minConf) # 对规则进行评估
        return bigRuleList

# steps 1 : Load IBM data to a list
def loadGenerateDataSet(filename = 'data.ntrans_0.1.nitems_0.01'):
    dataset = pd.read_csv(filename)
    dataset.columns = ['items']
    
    num_trans = []
    for patterns in dataset['items']:
        num_trans.append(int(patterns.split()[0]))
    
    dataset['Code'] = num_trans

    for i in range(len(dataset['items'])):
        dataset['items'][i] = int(dataset['items'][i].split()[2])

    # make every transaction in each list which is contained by a big list
    dataset_list = []
    for i in range(dataset['Code'].nunique()):
        dataset_list.append(list(dataset['items'][dataset['Code'] == i+1]))
    
    #D is a dataset in the setform.
    D = list(map(set,dataset_list))
    return D

if __name__ == '__main__':
    
    # load data when I create by IBM Quest Synthetic Data Generator
    dataSet = loadGenerateDataSet()
    # implementation Algorithm
    ap = Apriori()
    
    # Assumption min support = 0.5
    L, suppData = ap.apriori(dataSet, minSupport=0.5) 
    print L
    print suppData
    # Assumption min confidence = 0.6
    rules = ap.generateRules(L, suppData, minConf=0.6)
    print rules

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[[frozenset([7]), frozenset([5]), frozenset([3]), frozenset([0]), frozenset([6]), frozenset([8]), frozenset([4]), frozenset([9])], [frozenset([5, 7]), frozenset([3, 7]), frozenset([3, 6]), frozenset([5, 6]), frozenset([6, 7]), frozenset([9, 6]), frozenset([8, 9]), frozenset([4, 7]), frozenset([0, 9]), frozenset([0, 6]), frozenset([0, 5]), frozenset([3, 5]), frozenset([9, 3]), frozenset([3, 4]), frozenset([9, 5]), frozenset([8, 4]), frozenset([9, 4]), frozenset([0, 4]), frozenset([8, 5]), frozenset([8, 3]), frozenset([0, 8]), frozenset([0, 3]), frozenset([0, 7]), frozenset([8, 7]), frozenset([4, 6]), frozenset([8, 6]), frozenset([9, 7]), frozenset([4, 5])], [frozenset([8, 6, 7]), frozenset([4, 6, 7]), frozenset([8, 9, 4]), frozenset([8, 4, 5]), frozenset([4, 5, 6]), frozenset([0, 9, 4]), frozenset([0, 3, 5]), frozenset([0, 3, 7]), frozenset([0, 8, 5]), frozenset([0, 8, 7]), frozenset([8, 3, 7]), frozenset([8, 5, 7]), frozenset([0, 9, 6]), frozenset([9, 3, 4]), frozenset([8, 4, 6]), froz

frozenset([8, 5]) --> frozenset([0, 6]) conf: 0.653333333333
frozenset([5, 6]) --> frozenset([0, 8]) conf: 0.816666666667
frozenset([0, 8]) --> frozenset([5, 6]) conf: 0.680555555556
frozenset([0, 6]) --> frozenset([8, 5]) conf: 0.875
frozenset([0, 5]) --> frozenset([8, 6]) conf: 0.790322580645
frozenset([6]) --> frozenset([0, 8, 5]) conf: 0.7
frozenset([5]) --> frozenset([0, 8, 6]) conf: 0.620253164557
frozenset([0]) --> frozenset([8, 5, 6]) conf: 0.662162162162
frozenset([8, 6]) --> frozenset([3, 5]) conf: 0.823529411765
frozenset([8, 5]) --> frozenset([3, 6]) conf: 0.746666666667
frozenset([5, 6]) --> frozenset([8, 3]) conf: 0.933333333333
frozenset([8, 3]) --> frozenset([5, 6]) conf: 0.651162790698
frozenset([3, 6]) --> frozenset([8, 5]) conf: 0.848484848485
frozenset([3, 5]) --> frozenset([8, 6]) conf: 0.736842105263
frozenset([8]) --> frozenset([3, 5, 6]) conf: 0.608695652174
frozenset([6]) --> frozenset([8, 3, 5]) conf: 0.8
frozenset([5]) --> frozenset([8, 3, 6]) conf: 0.7088607

frozenset([8, 9, 3]) --> frozenset([0, 6]) conf: 0.632911392405
frozenset([9, 3, 6]) --> frozenset([0, 8]) conf: 0.806451612903
frozenset([8, 3, 6]) --> frozenset([0, 9]) conf: 0.78125
frozenset([0, 8, 9]) --> frozenset([3, 6]) conf: 0.769230769231
frozenset([0, 9, 6]) --> frozenset([8, 3]) conf: 0.961538461538
frozenset([0, 8, 6]) --> frozenset([9, 3]) conf: 0.892857142857
frozenset([0, 9, 3]) --> frozenset([8, 6]) conf: 0.78125
frozenset([0, 8, 3]) --> frozenset([9, 6]) conf: 0.746268656716
frozenset([0, 3, 6]) --> frozenset([8, 9]) conf: 0.943396226415
frozenset([8, 9]) --> frozenset([0, 3, 6]) conf: 0.602409638554
frozenset([9, 6]) --> frozenset([0, 8, 3]) conf: 0.769230769231
frozenset([8, 6]) --> frozenset([0, 9, 3]) conf: 0.735294117647
frozenset([3, 6]) --> frozenset([0, 8, 9]) conf: 0.757575757576
frozenset([0, 9]) --> frozenset([8, 3, 6]) conf: 0.746268656716
frozenset([0, 6]) --> frozenset([8, 9, 3]) conf: 0.892857142857
frozenset([0, 8]) --> frozenset([9, 3, 6]) conf: 0.694